In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
import pandas as pd
from tdt import read_block, read_sev, epoc_filter 
matplotlib.rcParams['font.size'] = 18 # set font size for all figures

In [2]:
%config Completer.use_jedi = False
%matplotlib inline
GCAMP = '_465N' # GCaMP channel (dynamic signal) 
ISOS = '_405N' 

In [3]:
BLOCK_PATH = '/data/gergely/invivo_DATA/fiberPhotometry/dock05b-1-210224-104500/EEGCamFP2-210312-104004/'
data = read_block(BLOCK_PATH)

Found Synapse note file: /data/gergely/invivo_DATA/fiberPhotometry/dock05b-1-210224-104500/EEGCamFP2-210312-104004/Notes.txt
read from t=0s to t=1693.63s


In [4]:
data.streams[GCAMP].data.shape

(1722752,)

In [4]:
def resample(data):
    """making a time array based on the number of samples 
    and the sample frequancy resampling
    
    Parameters:
    -----------
    data : tdt block"""
    
    npts = len(data.streams[GCAMP].data)
    time_x = np.linspace(1, npts, npts) / data.streams[GCAMP].fs
    
    return time_x

In [5]:
def downsampling(data, channel, N=10):
    """downsampling the data for plottig
    
    Parameters:
    -----------
    data : tdt block
    channel : global variable
        either GCAMP or ISOS
    N : int
        number of points for averaging
        
    Returns:
    --------
    dictionary of numpy arrays with the decimated signal and time 
    """
    decimatedData = {}
    decimatedSignal = []
    decimatedTime = []
            
    for i in range(0, len(data.streams[channel].data), N):        
        # This is the moving window mean
        mean_wnd = np.mean(data.streams[channel].data[i:i+N-1])
        decimatedSignal.append(mean_wnd)
    np.array(decimatedSignal)
    
    time_x = resample(data)
    time_x = time_x[::N] # go from beginning to end of array in steps on N
    time_x = time_x[:len(data.streams[channel].data)]
    
    decimatedData['decimatedSignal'] = decimatedSignal
    decimatedData['decimatedTime'] = time_x
    
    return decimatedData 

In [8]:
def artifactRemoval(data):
    """Artifact Removal There is often a large artifact on the onset
       of LEDs turning on Remove data below a set time t
       
    Parameters:
    -----------
    data : tdt block
    channel : global variable
        either GCAMP or ISOS
     """
    noArtifactData = {}
    noArtifactGAMP = []
    noArtifactISOS = []
    t = 8
    time_x = resample(data)
    
    inds = np.where(time_x > t)
    ind = inds[0][0]
    time_x = time_x[ind:] # go from ind to final index
    
    noArtifactData['GCAMP'] = data.streams[GCAMP].data[ind:]
    noArtifactData['ISOS'] = data.streams[ISOS].data[ind:]
    noArtifactData['TIME'] = time_x
    
    return noArtifactData

In [9]:
def detrending(data):
    """Full trace dFF according to Lerner et al. 2015
       dFF using 405 fit as baseline
    """
    x = artifactRemoval(data)['ISOS']
    y = artifactRemoval(data)['GCAMP']
    bls = np.polyfit(x, y, 1)
    Y_fit_all = np.multiply(bls[0], x) + bls[1]
    Y_dF_all = y - Y_fit_all

    dFF = np.multiply(100, np.divide(Y_dF_all, Y_fit_all))
    std_dFF = np.std(dFF)
    
    return dFF

In [10]:
detrending(data)

/home/gergely/.virtualenvs/photometry/lib/python3.6/site-packages/ipykernel_launcher.py:1: RankWarning: Polyfit may be poorly conditioned
  """Entry point for launching an IPython kernel.


array([-0.08585075, -0.08331067, -0.07889117, ..., -0.33919498,
       -0.31118608, -0.28058457], dtype=float32)

In [12]:
iso

array([104.82309 , 104.80421 , 104.78272 , ..., 101.50839 , 101.47111 ,
       101.428246], dtype=float32)

In [11]:
down = downsampling(data, GCAMP)

In [18]:
down['decimatedSignal']

[0.2549811,
 0.18692806,
 0.08100607,
 0.15060447,
 0.3438492,
 3.7350667,
 16.25793,
 44.9622,
 88.96597,
 133.5123,
 158.56827,
 156.48668,
 139.13908,
 125.99588,
 126.34485,
 134.58531,
 140.3694,
 139.6202,
 135.84323,
 133.83768,
 134.62341,
 136.05124,
 136.28944,
 135.58508,
 135.14276,
 135.4525,
 136.0084,
 136.0794,
 135.5508,
 134.95029,
 134.90308,
 135.44179,
 136.11615,
 136.4625,
 136.41733,
 136.22018,
 136.00395,
 135.8519,
 135.73236,
 135.61305,
 135.46082,
 135.39938,
 135.58002,
 135.8661,
 136.05461,
 135.98082,
 135.79677,
 135.64842,
 135.67023,
 135.80865,
 135.95406,
 135.92064,
 135.6776,
 135.40143,
 135.32303,
 135.50264,
 135.77747,
 136.0077,
 136.14545,
 136.28587,
 136.52562,
 136.77728,
 136.87366,
 136.68361,
 136.33444,
 135.95892,
 135.70943,
 135.6709,
 135.74002,
 135.82938,
 135.85524,
 135.82503,
 135.7962,
 135.80273,
 135.84471,
 135.82663,
 135.75642,
 135.689,
 135.72574,
 135.84633,
 135.9885,
 136.13959,
 136.28209,
 136.41675,
 136.40659

In [43]:
decDat = {}

In [44]:
decDat['signal'] = down

In [46]:
type(decDat)

dict

In [22]:
import sys
try:
    def foo():
        print('magn')
foo()
except NameError as e:
print e
print sys.exc_type


SyntaxError: invalid syntax (<ipython-input-22-6a8e8be9820e>, line 5)